# Rokh Star

## Code analysis : 

Analysing the code we found that the only way to decode the flag is by finding the output of this function that overflows the stack
```python
@functools.cache
def gen_function(i):
    if i == 0: return 1
    if i == 1: return 2
    if i == 2: return 3
    if i == 3: return 4
    return -44574*gen_function(i-4) +5767*gen_function(i-3) + 427*gen_function(i-2) - 19*gen_function(i-1)
```

We have to find a new way to find `gen_function(ITERS = int(6e7))` so we could get the key and decrypt the file <br>
The function is a polynomial of degree 4, so we can use **Linear Algebra** to solve it <br>
How ? Let's figure it out

## Define Matrice :

Let's have $Y_{n}$ such :
$$
gen\_function(n) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
\end{pmatrix} * Y_{n} \begin{pmatrix}
gen\_function(n)\\ 
gen\_function(n+1)\\
gen\_function(n+2)\\
gen\_function(n+3)
\end{pmatrix}
$$

So we have to write  $ Y_{n+1} = M * Y_{n} $  with :
$$ 
M = \begin{pmatrix}
0 & 1 & 0 & 0\\ 
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
-44574 & 5767 & 427 & -19
\end{pmatrix}  
$$

#### Recursively :

$$
    Y_{n+1} = M * Y_{n}  = M * M * Y_{n-1} = ..... = M^{n+1} * Y_0 
    \begin{pmatrix}
1\\ 
2\\
3\\
4
\end{pmatrix}
$$
#### Putting all together :
$$ gen\_function(n) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
\end{pmatrix} * M^{n} * Y_0
$$

### But what ?

Calculating $M^{n}$ is as hard and complex as calculating directely `gen_function(n)` so what to do with all that ?

Although sageMath can calculate that rapidly, It is not a good solution for bigger matrices .

## This is why we use linear algebra : Diagonal Matrice

Let's have a matrice 
$$
M = P * D \begin{pmatrix}
\alpha_1 & 0 & 0 & 0\\ 
0 & \alpha_2 & 0 & 0\\
0 & 0 & \alpha_3 & 0\\
0 & 0 & 0 & \alpha_4
\end{pmatrix}   
* P^{-1}
$$
So calculating  M would be easy since 
$$
M^{n} = P * D^{n} \begin{pmatrix}
\alpha_1^n & 0 & 0 & 0\\ 
0 & \alpha_2^n & 0 & 0\\
0 & 0 & \alpha_3^n & 0\\
0 & 0 & 0 & \alpha_4^n
\end{pmatrix} * P^{-1}
$$

#### Putting all together :
$$ gen\_function(n) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
\end{pmatrix} * P * D^{n} * P^{-1} * Y_0
$$

## Let's Code :

In [1]:
from sympy import *
M=Matrix([[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [-44574, 5767, 427, -19]])
P,D = M.diagonalize()
Pi=P**-1
L=Matrix([[1,0,0,0]])*P 
R=Pi*Matrix([[1],[2],[3],[4]])
n = Symbol('n')
f = (L * D**n * R)[0]
print(f"gen_function(n) = {f}")


gen_function(n) = 187*(-19)**n/450 - 63*(-23)**n/232 - 107*17**n/792 + 7904*6**n/7975


Let's get the GCD and make it more beautiful function

In [2]:
g = 1/gcd(tuple(R))
R = g*R
newf = (L * D**n * R)[0]
print(f'Formatted gen_function(n) = ({newf}) // {g}')

Formatted gen_function(n) = (238612*(-19)**n - 155925*(-23)**n - 77575*17**n + 569088*6**n) // 574200


Finally we have to use `mpz(n)` for easier computation

In [3]:
from gmpy2 import mpz
def newFunction(n):
    f = (238612*mpz(-19)**n - 155925*mpz(-23)**n - 77575*mpz(17)**n + 569088*mpz(6)**n) // 574200
    return f


In [2]:
from gmpy2 import mpz
ITERS = int(6e7)
def newFunction(n):
    f = (238612*mpz(-19)**n - 155925*mpz(-23)**n - 77575*mpz(17)**n + 569088*mpz(6)**n) // 574200
    return f

print(str(newFunction(ITERS))[:30])

-39346683987984416804020351651


## Get Function and put all together

In [2]:
from Crypto.Util.number import long_to_bytes
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib
import functools
from gmpy2 import mpz

ITERS = int(6e7)
def xor(a:bytes,b:bytes):
    s = b''
    for (i,j) in zip(a,b):
        s+=long_to_bytes(i^j)
    return s
def newFunction(n):
    f = (238612*mpz(-19)**n - 155925*mpz(-23)**n - 77575*mpz(17)**n + 569088*mpz(6)**n) // 574200
    return f


def gen_key(k):
    k = str(k)
    k1,k2 = k[:len(k)//2],k[len(k)//2+1:]
    k1 = hashlib.sha256(k1.encode()).digest()
    k2 = hashlib.sha256(k2.encode()).digest()
    k = xor(k1,k2)
    return k[:16],k[16:]
k = newFunction(ITERS)
iv,k =gen_key(k)
aes = AES.new(key=k,iv=iv,mode=AES.MODE_CBC)
with open('out.txt','rb') as f :
    enc = f.read()
f.close()
enc = aes.decrypt(enc)
enc = unpad(enc,16)
print(enc)

b'Securinets{l1n3ar_aLGbr4_f0R_th3_WIN}'
